In [18]:
import pandas as pd
import numpy as np
# import math

# from datetime import date
# from datetime import timedelta

# import datetime

from os.path import exists as file_exists
from openpyxl import load_workbook

In [19]:
in_file = "Materialmasterdata.xlsx"
status_sheet = "Report"
active_products = "ZP02"
pl_plant_sheet = "PL_Plant"
# stat_definitions = 'Stat_definitions'
stat_cross_ref = 'Stat_cross_ref'

out_file = "Status_verification.xlsx"
print(f'From {in_file} take:\n',
      'statuses from {status_sheet}\n',
      f'active products from{active_products}\n',
      f'statuses corelation from {stat_cross_ref}\n',
      f'delivering plant for PL from {pl_plant_sheet}')

From Materialmasterdata.xlsx take:
 statuses from {status_sheet}
 active products fromZP02
 statuses corelation from Stat_cross_ref
 delivering plant for PL from PL_Plant


In [20]:
def read_in_file(in_file, in_SheetName = '', header_row = 0):
    
    try:
        wb = pd.read_excel(in_file, sheet_name = None, header = None)
        sheets = list(wb.keys())
        if in_SheetName == '':
            print ("Loading last sheet in file")
            df = wb.get(sheets[-1])
        else:
            df = wb.get(in_SheetName)
        
        del(wb)                
    
    except TypeError:
        wb = load_workbook(filename = in_file, data_only = True)
        
        if in_SheetName == '':
            print ("Loading last sheet in file")
            df = pd.DataFrame(wb[wb.sheetnames[-1]].values)
            
        else:
            df = pd.DataFrame(wb[in_SheetName].values)

        sheets = wb.sheetnames
        wb.close()
        
    except FileNotFoundError:
        df = pd.DataFrame()
        sheets=[]
        print(f'File {in_file} not found')
    
    except:
        print("oops!")
        pass

    column_names = rename_columns(df.iloc[header_row].values)
    df.columns = column_names
    df.drop(index = [i for i in range(header_row+1)], inplace = True)
    df.reset_index(drop = True, inplace = True)

    return df, sheets


def rename_columns(column_names):
    for item in enumerate(column_names):
        try:
            column_names[item[0]] = '_'.join(item[1].split())
        except:
            column_names[item[0]] = item[1]
    return column_names

In [21]:
%%time

df_statuses, sheets = read_in_file(in_file, status_sheet, header_row = 0)
df_products, _ = read_in_file(in_file, active_products, header_row = 0)
df_pl_plant, _ = read_in_file(in_file, pl_plant_sheet, header_row = 0)
df_stats_ref, _ = read_in_file(in_file, stat_cross_ref, header_row = 0)
print (sheets)


['_com.sap.ip.bi.xl.hiddensheet', 'Report', 'ZP02', 'PL_Plant', 'Stat_definitions', 'Stat_cross_ref']
Wall time: 20.7 s


In [22]:
df_statuses = df_statuses[['Material', 'Plant', 'Product_line', 'X-distr.chain_status', 'DChain_Status']]
df_statuses

,Material,Plant,Product_line,X-distr.chain_status,DChain_Status
0,109007,A101,21,51,51
1,109026,A101,21,51,40
2,109030,A101,21,51,51
3,7701021,A101,17,51,51
4,7701021,A201,17,51,51
...,...,...,...,...,...
17530,SZ185-4RI,A201,17,51,51
17531,SZ185-9RI,A101,17,51,57
17532,SZ185-9RI,A201,17,51,57
17533,US2810BX,A101,42,51,57


In [23]:
df_products = df_products[['Material_Number', 'Text_Material', 'Amount_(Currency)']]
# if several prices with different validity dates exists - take first record
df_products.drop_duplicates(subset='Material_Number', keep='first', inplace=True)
df_products.set_index('Material_Number', inplace=True)
# df_products['Amount_(Currency)'] = df_products['Amount_(Currency)'].replace(0, np.nan)
df_products

,Text_Material,Amount_(Currency)
Material_Number,,
061H3003,RGE Fan Speed Controller Z3R4-7DS,30470
061H3005,RGE Fan Speed Controller Z1N4-7DS,7370
061H3006,RGE Fan Speed Controller X3R4-7DS,35720
061H3008,RGE Fan Speed Controller Z1P4-7DS,10320
061H3009,RGE Fan Speed Controller Z1Q4-7DS,12750
...,...,...
145B0003,EvoFlat FSS 2 TPC-M,53160
145B0077,"EvoFlat FSS FR1 TPC-M TWA w/Cover, BP",48920
267B0154,Akva Lux ID1 Romstal,100000


In [24]:
df_statuses = df_statuses.join(df_products, on='Material', how='outer').reset_index(drop=True)

df_statuses

,Material,Plant,Product_line,X-distr.chain_status,DChain_Status,Text_Material,Amount_(Currency)
0,109007,A101,21,51,51,Connection kit for PT heating cables,922
1,109026,A101,21,51,40,NaN,NaN
2,109030,A101,21,51,51,"Montagestege TYPE MS 5,6-6,5 MM - 25MM",56
3,7701021,A101,17,51,51,START KIT-CSRnotUL 10+15+100µF MT/Z 18-1,9819
4,7701021,A201,17,51,51,START KIT-CSRnotUL 10+15+100µF MT/Z 18-1,9819
...,...,...,...,...,...,...,...
17595,005U0297,NaN,NaN,NaN,NaN,S22 GASKET EPDM SL,268
17596,079B6045,NaN,NaN,NaN,NaN,Gasket EPDM HT Hang on for S31A,223
17597,079B6654,NaN,NaN,NaN,NaN,S14A-ST16-21-TKTL90,46590
17598,145B0003,NaN,NaN,NaN,NaN,EvoFlat FSS 2 TPC-M,53160


In [25]:
df_stats_ref.set_index(df_stats_ref.columns[0], inplace=True)

df_stats_ref['B600_accepted'] = df_stats_ref.transpose().apply(lambda x: x.dropna().index.to_list())
df_stats_ref = df_stats_ref[['B600_accepted']]
df_stats_ref

,B600_accepted
Code,
47,[57]
51,"[51, 57]"
52,"[51, 52, 57]"
53,"[51, 53, 57]"
54,"[51, 54, 57]"
55,"[51, 55, 57]"
61,"[51, 61, 57]"
63,"[51, 63, 57]"
72,[57]


In [26]:
df_statuses[['X-distr.chain_status', 'DChain_Status']]=df_statuses[['X-distr.chain_status', 'DChain_Status']].replace(np.nan,0)
df_statuses = df_statuses.astype({'X-distr.chain_status':'int64', 'DChain_Status':'int64'})
df_statuses = df_statuses.join(df_stats_ref, on='X-distr.chain_status', how ='left')

df_statuses['B600_accepted']=df_statuses['B600_accepted'].apply(lambda x: x if isinstance(x, list) else [])
df_statuses

,Material,Plant,Product_line,X-distr.chain_status,DChain_Status,Text_Material,Amount_(Currency),B600_accepted
0,109007,A101,21,51,51,Connection kit for PT heating cables,922,"[51, 57]"
1,109026,A101,21,51,40,NaN,NaN,"[51, 57]"
2,109030,A101,21,51,51,"Montagestege TYPE MS 5,6-6,5 MM - 25MM",56,"[51, 57]"
3,7701021,A101,17,51,51,START KIT-CSRnotUL 10+15+100µF MT/Z 18-1,9819,"[51, 57]"
4,7701021,A201,17,51,51,START KIT-CSRnotUL 10+15+100µF MT/Z 18-1,9819,"[51, 57]"
...,...,...,...,...,...,...,...,...
17595,005U0297,NaN,NaN,0,0,S22 GASKET EPDM SL,268,[]
17596,079B6045,NaN,NaN,0,0,Gasket EPDM HT Hang on for S31A,223,[]
17597,079B6654,NaN,NaN,0,0,S14A-ST16-21-TKTL90,46590,[]
17598,145B0003,NaN,NaN,0,0,EvoFlat FSS 2 TPC-M,53160,[]


In [27]:
# df_pl_plant.rename(columns={'Division':'key'}, inplace=True)
df_pl_plant.set_index(df_pl_plant.columns[0], inplace=True)
df_pl_plant.dropna(how='all', inplace = True)
df_pl_plant['Plant_accepted'] = df_pl_plant.transpose().apply(lambda x: x.dropna().index.to_list())
df_pl_plant = df_pl_plant[['Plant_accepted']]
df_pl_plant

,Plant_accepted
Division,
01,[A101]
03,[A101]
04,[A101]
06,[A201]
07,[A101]
08,[A101]
14,[A101]
17,[A201]
21,[A101]


In [28]:
df_statuses = df_statuses.join(df_pl_plant, on='Product_line', how='left')
df_statuses['Plant_accepted'] = df_statuses['Plant_accepted'].apply(lambda x: x if isinstance(x, list) else [])

df_statuses

,Material,Plant,Product_line,X-distr.chain_status,DChain_Status,Text_Material,Amount_(Currency),B600_accepted,Plant_accepted
0,109007,A101,21,51,51,Connection kit for PT heating cables,922,"[51, 57]",[A101]
1,109026,A101,21,51,40,NaN,NaN,"[51, 57]",[A101]
2,109030,A101,21,51,51,"Montagestege TYPE MS 5,6-6,5 MM - 25MM",56,"[51, 57]",[A101]
3,7701021,A101,17,51,51,START KIT-CSRnotUL 10+15+100µF MT/Z 18-1,9819,"[51, 57]",[A201]
4,7701021,A201,17,51,51,START KIT-CSRnotUL 10+15+100µF MT/Z 18-1,9819,"[51, 57]",[A201]
...,...,...,...,...,...,...,...,...,...
17595,005U0297,NaN,NaN,0,0,S22 GASKET EPDM SL,268,[],[]
17596,079B6045,NaN,NaN,0,0,Gasket EPDM HT Hang on for S31A,223,[],[]
17597,079B6654,NaN,NaN,0,0,S14A-ST16-21-TKTL90,46590,[],[]
17598,145B0003,NaN,NaN,0,0,EvoFlat FSS 2 TPC-M,53160,[],[]


In [29]:
# statuses are the same?
df_statuses['Is_X-DC_equal'] = df_statuses['X-distr.chain_status'] == df_statuses['DChain_Status']

# is current DC_status within accepted for B600 variants?
df_statuses['Is_Status_accepted'] = df_statuses[['DChain_Status', 'B600_accepted']].apply(
    lambda row: row[0] in row[1], axis=1)

# is current DC_status 'blocked' if no price?
df_statuses['Is_Status_accepted'] = df_statuses[['Is_Status_accepted', 'DChain_Status', 'Amount_(Currency)']].apply(
    lambda row: row[0] if row[2] else row[1] in [57, 78], axis=1)

# is current Plant within accepted for B600 variants?
df_statuses['Is_Plant_correct'] = df_statuses[['Plant', 'Plant_accepted']].apply(lambda row: row[0] in row[1], axis=1)

df_statuses

,Material,Plant,Product_line,X-distr.chain_status,DChain_Status,Text_Material,Amount_(Currency),B600_accepted,Plant_accepted,Is_X-DC_equal,Is_Status_accepted,Is_Plant_correct
0,109007,A101,21,51,51,Connection kit for PT heating cables,922,"[51, 57]",[A101],True,True,True
1,109026,A101,21,51,40,NaN,NaN,"[51, 57]",[A101],False,False,True
2,109030,A101,21,51,51,"Montagestege TYPE MS 5,6-6,5 MM - 25MM",56,"[51, 57]",[A101],True,True,True
3,7701021,A101,17,51,51,START KIT-CSRnotUL 10+15+100µF MT/Z 18-1,9819,"[51, 57]",[A201],True,True,False
4,7701021,A201,17,51,51,START KIT-CSRnotUL 10+15+100µF MT/Z 18-1,9819,"[51, 57]",[A201],True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
17595,005U0297,NaN,NaN,0,0,S22 GASKET EPDM SL,268,[],[],True,False,False
17596,079B6045,NaN,NaN,0,0,Gasket EPDM HT Hang on for S31A,223,[],[],True,False,False
17597,079B6654,NaN,NaN,0,0,S14A-ST16-21-TKTL90,46590,[],[],True,False,False
17598,145B0003,NaN,NaN,0,0,EvoFlat FSS 2 TPC-M,53160,[],[],True,False,False


In [30]:
df_plant_ok = df_statuses[df_statuses['Is_Plant_correct']==True]
df_plant_wrong = df_statuses.drop(df_plant_ok.index)

# from df_plant_wrong remove products that exist in df_plant_ok
df_plant_wrong = df_plant_wrong.drop(
    df_plant_wrong[df_plant_wrong['Material'].isin(df_plant_ok['Material'].unique())].index)

# from df_plant_wrong remove products without price
df_plant_wrong = df_plant_wrong[~df_plant_wrong['Amount_(Currency)'].isna()]

df_plant_wrong

,Material,Plant,Product_line,X-distr.chain_status,DChain_Status,Text_Material,Amount_(Currency),B600_accepted,Plant_accepted,Is_X-DC_equal,Is_Status_accepted,Is_Plant_correct
1675,004B1368,A101,48,72,72,"IZB-Plates for XG31L, I-pack 10 pcs",3659,[57],"[A113, A115]",True,False,False
1676,004B1370,A101,48,72,72,"IZB-Plates for XG31H, I-pack 10 pcs",3659,[57],"[A113, A115]",True,False,False
1987,004B2930,A101,48,72,51,"XG10: spec. plates 1-pass, incl.gaskets",2653,[57],"[A113, A115]",False,False,False
1988,004B2931,A101,48,72,51,"XG10: spec. plates 2-pass, incl.gaskets",3502,[57],"[A113, A115]",False,False,False
2089,004B6931,A101,48,72,51,"Gasket for XG10, I-pack 10 pcs",497,[57],"[A113, A115]",False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
17595,005U0297,NaN,NaN,0,0,S22 GASKET EPDM SL,268,[],[],True,False,False
17596,079B6045,NaN,NaN,0,0,Gasket EPDM HT Hang on for S31A,223,[],[],True,False,False
17597,079B6654,NaN,NaN,0,0,S14A-ST16-21-TKTL90,46590,[],[],True,False,False
17598,145B0003,NaN,NaN,0,0,EvoFlat FSS 2 TPC-M,53160,[],[],True,False,False


In [31]:
df_output = pd.concat([df_plant_wrong, df_plant_ok])
df_output

,Material,Plant,Product_line,X-distr.chain_status,DChain_Status,Text_Material,Amount_(Currency),B600_accepted,Plant_accepted,Is_X-DC_equal,Is_Status_accepted,Is_Plant_correct
1675,004B1368,A101,48,72,72,"IZB-Plates for XG31L, I-pack 10 pcs",3659,[57],"[A113, A115]",True,False,False
1676,004B1370,A101,48,72,72,"IZB-Plates for XG31H, I-pack 10 pcs",3659,[57],"[A113, A115]",True,False,False
1987,004B2930,A101,48,72,51,"XG10: spec. plates 1-pass, incl.gaskets",2653,[57],"[A113, A115]",False,False,False
1988,004B2931,A101,48,72,51,"XG10: spec. plates 2-pass, incl.gaskets",3502,[57],"[A113, A115]",False,False,False
2089,004B6931,A101,48,72,51,"Gasket for XG10, I-pack 10 pcs",497,[57],"[A113, A115]",False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
17526,SZ185-3RI,A201,17,51,57,SZ185-3RC-SINGLE,0,"[51, 57]",[A201],False,True,True
17528,SZ185-4CAI,A201,17,51,51,SZ185-4CD-SINGLE,189600,"[51, 57]",[A201],True,True,True
17530,SZ185-4RI,A201,17,51,51,SZ185-4RD-SINGLE,189600,"[51, 57]",[A201],True,True,True
17532,SZ185-9RI,A201,17,51,57,SZ185-9RC-SINGLE,0,"[51, 57]",[A201],False,True,True


In [32]:
if file_exists(out_file):
    print(f'Replacing {out_file}\n')
else:
    print(f'Creating {out_file}\n')
    
with pd.ExcelWriter(out_file, mode = 'w') as writer:
    df_output.to_excel(writer, sheet_name = 'Total', index=False)    
#     df_concat.to_excel(writer, sheet_name = 'Transformed_duplicates', index=False)    
    
    writer.save()


Creating Status_verification.xlsx



In [33]:
# df_noprice = df_plant_ok[df_plant_ok['Amount_(Currency)'].isna()]
# df_noprice = pd.concat([df_noprice, df_plant_ok[df_plant_ok['Amount_(Currency)']==0]]).sort_index()
# df_price_ok = df_plant_ok.drop(df_noprice.index)
# df_price_ok
# # df_noprice

In [34]:
# df_price_ok[df_price_ok['Is_Status_accepted']==True] # active and OK

In [35]:
# df_price_ok[df_price_ok['Is_Status_accepted']==False] # active, status wrong

In [36]:
# df_noprice[df_noprice['Is_Status_accepted']==False] # inactive, status wrong

In [37]:
# df_noprice[df_noprice['Is_Status_accepted']==True] # inactive, everything OK

In [44]:
df_output['Material'].value_counts().head(20)

005U0259    2
188X7331    2
134Z0009    2
134Z0001    2
010H0153    2
010H0156    2
010H0159    2
010H0179    2
010H0336    2
010H0338    2
030U1497    2
030U1494    2
030U1489    2
030U1362    2
030U1355    2
030U1352    2
030U1345    2
030U1318    2
030U1315    2
030U1313    2
Name: Material, dtype: int64